In [1]:
# -*- coding: utf-8 -*-
"""
Created on Thu Feb 20 13:18:31 2025

@author: nelso
"""

import numpy as np
import plotly.graph_objects as go
import plotly.io as pio
from scipy.special import comb
import dash
from dash import dcc, html
from dash.dependencies import Input, Output
import webbrowser

#Function to calculate the probability to have exactly ka A and exactly kb B
def prob_exactly_ka_and_exactly_kb(KA, KB, total_cards, hand_size, ka, kb):
    total_hands = comb(total_cards, hand_size)
    other_cards = total_cards - KA - KB
    
    valid_mask = (KA >= ka) & (KB >= kb) & (other_cards >= (hand_size - ka - kb))
    prob = np.zeros_like(KA, dtype=float)
    prob[valid_mask] = (
        comb(KA[valid_mask], ka) *
        comb(KB[valid_mask], kb) *
        comb(other_cards[valid_mask], hand_size - ka - kb)
    ) / total_hands
    
    return prob

#We use dash
app = dash.Dash(__name__)

#Layout of the application
app.layout = html.Div([
    html.H1("Distribution of Probabilities - exactly ka and exactly kb (3D)"),
    dcc.Graph(id='3d-plot', style={'height': '800px'}),  # just to upgrade the size of the graph
    html.Label("Total Cards"),
    dcc.Slider(id='total-cards', min=40, max=60, step=1, value=40, marks={i: str(i) for i in range(40, 61, 5)}),
    html.Label("Hand Size"),
    dcc.Slider(id='hand-size', min=0, max=60, step=1, value=5, marks={i: str(i) for i in range(0, 61, 5)}),
    html.Label("ka"),
    dcc.Slider(id='ka', min=0, max=5, step=1, value=2, marks={i: str(i) for i in range(0, 6)}),
    html.Label("kb"),
    dcc.Slider(id='kb', min=0, max=5, step=1, value=1, marks={i: str(i) for i in range(0, 6)})
])

# Callback to update the graph
@app.callback(
    Output('3d-plot', 'figure'),
    [Input('total-cards', 'value'),
     Input('hand-size', 'value'),
     Input('ka', 'value'),
     Input('kb', 'value')]
)
def update_plot(total_cards, hand_size, ka, kb):
    KA_values = np.arange(0, total_cards + 1)
    KB_values = np.arange(0, total_cards + 1)
    KA, KB = np.meshgrid(KA_values, KB_values)
    Z = prob_exactly_ka_and_exactly_kb(KA, KB, total_cards, hand_size, ka, kb)
    
    fig = go.Figure(data=[go.Surface(z=Z, x=KA_values, y=KB_values, colorscale='Viridis')])
    fig.update_layout(
        scene=dict(
            xaxis_title='Number of copies of A (KA)',
            yaxis_title='Number of copies of B (KB)',
            zaxis_title=f'Probability of drawing exactly {ka} A and exactly {kb} B',
        ),
        title=f'Evolution of P(exactly {ka} A and exactly {kb} B) depending on KA and KB',
        width=1000,  #to upgrade the weigth
        height=800   #to upgrade the heigth
    )
    return fig


#Open page automatically in browser
if __name__ == '__main__':
    webbrowser.open('http://127.0.0.1:8051')
    app.run(debug=True, port=8051)
